In [1]:
import numpy as np
import pandas as pd
import geopy.distance
from tqdm import tqdm

In [5]:
#df = pd.read_csv('../DABA/middle_관광지위치.xlsx', encoding='euc-kr')
df = pd.read_excel('../DABA/middle_관광지위치.xlsx')
df

,경도,위도,중심시도명,중심시군구명,중심관광지명,Unnamed: 5,중심카테고리 명_대,중심카테고리 명_중,순위
0,127.0591,37.5120,서울특별시,강남구,코엑스,서울 강남구 영동대로 513-0,관광지,문화관광,1
1,127.0597,37.5087,서울특별시,강남구,현대백화점/무역센터점,서울 강남구 테헤란로 517-0,관광지,쇼핑,2
2,127.0275,37.5273,서울특별시,강남구,현대백화점/압구정본점,서울 강남구 압구정로 165-0,관광지,쇼핑,3
3,127.0416,37.5279,서울특별시,강남구,갤러리아백화점/명품관EAST,서울 강남구 압구정로 407-0,관광지,쇼핑,4
4,127.0400,37.5284,서울특별시,강남구,갤러리아백화점/명품관WEST,서울 강남구 압구정로 343-0,관광지,쇼핑,5
...,...,...,...,...,...,...,...,...,...
1731,127.0121,37.5672,서울특별시,중구,DDP패션몰,서울 중구 마장로 22-0,관광지,쇼핑,93
1732,126.9805,37.5620,서울특별시,중구,화폐박물관,서울 중구 남대문로 39-0,관광지,문화관광,94
1733,127.0075,37.5694,서울특별시,중구,평화시장,서울 중구 청계천로 274-0,관광지,쇼핑,95
1734,126.9792,37.5548,서울특별시,중구,남산백범광장,서울 중구,관광지,문화관광,98


In [7]:
subway = pd.read_csv('../../naver_crw/DABA/서울시 역사마스터 정보.csv', encoding='euc-kr')

In [8]:
# 프라이머리 키 생성
subway['subway_pk'] = np.nan
for i in range(subway.shape[0]):
    #print(subway['역사명'][i], subway['호선'][i])
    subway['subway_pk'][i] = f"{subway['역사명'][i]}, {subway['호선'][i]}"  

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_13320\2159356481.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subway['subway_pk'][i] = f"{subway['역사명'][i]}, {subway['호선'][i]}"


In [ ]:
# .. 가까운 지하철 거리 및 도보 시간 구하기
# 가까운 지하철 구하기

large_dist = []

#for i in tqdm(range(df.shape[0])):
for i in range(10):
    dummy = []

    if np.isnan(df.iloc[i]['위도']) :

        dummy.append(np.nan)
        find_min_j = np.nan
        find_stn_nam = np.nan
        find_stn_lat = np.nan
        find_stn_lon = np.nan
        nearest_stn = np.nan
        time_to_near = np.nan

    else:

        for j in range(subway.shape[0]):

            hotel_latlon = (df.iloc[i]['위도'], df.iloc[i]['경도'])
            subway_latlon = (subway.iloc[j]['경도'], subway.iloc[j]['위도'])    # 원본데이터에서 위경도 컬럼명 바뀌어 있음

            dist = geopy.distance.geodesic(hotel_latlon, subway_latlon).m
            dummy.append(dist)

        find_min_j = np.argmin(dummy)
        dist_to_stn = np.nanmin(dummy)
        time_to_stn = np.nanmin(dummy)/66.7

        #find_stn_nam = subway.iloc[find_min_j]['역사명']
        find_stn_nam = subway.iloc[find_min_j]['subway_pk']
        find_stn_lat = subway.iloc[find_min_j]['위도']
        find_stn_lon = subway.iloc[find_min_j]['경도']

        
            
        #print(f"호텔:{df.iloc[i]['사업장명']} {df.iloc[i]['지번주소']} {min(dummy)}m")
    large_dist.append([df.iloc[i]['중심관광지명'],
                        df.iloc[i]['Unnamed: 5'],
                        df.iloc[i]['위도'],
                        df.iloc[i]['경도'],
                        find_stn_nam,
                        find_stn_lat,
                        find_stn_lon,
                        dist_to_stn,
                        time_to_stn]) # 시속 4km --> 분속 66.7m


In [ ]:
find_large_dist = pd.DataFrame(large_dist, columns=['중심관광지명','주소','관광지 위도', '관광지 경도','가까운 지하철역', '역 위도','역 경도', '거리','도보시간(분)'])

#for i in range(10):
#    print(f"호텔:{large_df.iloc[i]['사업장명']}, {df.iloc[i]['거리']}")
find_large_dist.info()
find_large_dist

In [ ]:
df['가까운 지하철역'] = find_large_dist['역이름']
df['지하철역과의 거리'] = find_large_dist['거리']
df['도보시간'] = find_large_dist['도보시간(분)']

In [ ]:
df.info()

In [ ]:
colum_names2 = ['경도',	'위도',	'중심시도명', '중심시군구명', '중심관광지명', 'Unnamed: 5', '중심카테고리 명_대', '중심카테고리 명_중', '순위','가까운 지하철역', '지하철역과의 거리', '도보시간']

df1 = df[colum_names2]
df1

In [ ]:
output_path = "../DAOU/middle_관광지위치_지하철역추가.csv"
df1.to_csv(output_path, encoding='euc-kr')

In [ ]:
df1.info()